In [1]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import genfromtxt
from matplotlib.pyplot import cm
import os, subprocess
from scipy.interpolate import make_smoothing_spline
#from scipy.interpolate._smoothing_spline import make_smoothing_spline

kb = 1.380649 * (10 ** -23) # Boltzmann constant in kelvin
T = 298.15 # Temperature in Kelvin

In [2]:
dictionary_convert_to_ua =        {"ALA":"ALA","ARG":"ARG","ASN":"ASN",
                      "ASP":"ASP","CYS":"CYS","GLN":"GLN","GLU":"GLU",
                      "ILE" :"ILE","LEU":"LEU","LYS":"LYS",
                      "MET":"MET","PHE" :"PHE","PRO":"PRO","SER":"SER", 
                      "THR":"THR","TRP" :"TRP","TYR":"TYR","VAL":"VAL",
                      "HSD":"HID","HSE" :"HIE","HSP":"HIP",
                      "CYM":"CYM","ASPP":"AAN","GLUP":"GAN"
                      }

files = ["ALA", "ARG", "ASN", "ASP", "ASPP", "CYM", "CYS", "GLN", "GLU", "GLUP", "HSD", "HSE", "HSP", "ILE", "LEU",
         "LYS", "MET", "PHE", "PRO", "SER", "THR", "TRP", "TYR", "VAL"
        ]

print(len(files))

#files = ["ALA"]

24


In [3]:
# Function to read .xvg files
def read_xvg(file_name):
    # Initialize lists to store the data
    data = []
    with open(file_name, 'r') as f:
        for line in f:
            # Skip comment lines starting with '#'
            if line.startswith(('#', '@')):
                continue
            # Read numeric data
            columns = line.split()
            data.append([float(i) for i in columns])
    
    # Convert the list into a NumPy array for easy handling
    return np.array(data)

In [4]:
# Loops over files you want to get values for, use dictionary with correct naming convention
radgyr = []
for file in files: 
    file_name = file + "/rdf.xvg"
    data = read_xvg(file_name)
    r = data[:, 0]  # Time data (first column)
    RDF = data[:, 1]  # Data of interest (second column)
    
    #Normalise
    long_range = RDF[(r > 2) & (r < 3)]
    RDF = RDF/np.mean(long_range)
    
    
    # Radius of gyration for each sim, get max  and then average over all sims
    gyrate = file + "/gyrate.xvg"
    gyrate_dat = read_xvg(gyrate)
    xgyr = gyrate_dat[:, 1]
    ygyr = gyrate_dat[:, 2]
    zgyr = gyrate_dat[:, 3]
    xgyr_mean = np.mean(xgyr)
    ygyr_mean = np.mean(ygyr)
    zgyr_mean = np.mean(zgyr)
    radgyr.append(np.max([xgyr_mean, ygyr_mean, zgyr_mean]))
    
    """
    # Plot Radial Distribution function
    plt.figure(figsize=(8, 6))
    plt.plot(r, RDF, color='r')
    plt.xlabel('r (nm)', fontsize=14)
    plt.ylabel('g(r)', fontsize=14)
    plt.xlim(0, 2.5)
    plt.title('{} Radial Distribution Function'.format(file), fontsize=16)
    plt.grid(True)
    #plt.legend()
    """
    
    
    # Now get PMF 
    #pmf = -np.log(data[:, 1])  # Data of interest (second column)
    SSD = []
    PMF = []
    
    # For normalising, take running average of last points and shift down
    
    #data[:,1] = -np.log( data[:,1])
    #pmf = data[:,1] - np.mean(data[-10:,1])
    #pmf = data[:,1] - data[-1,1]
    #pmf = data[:,1]
    pmf = -np.log(RDF)

    
    # if normalising have data, else use pmf 
    for i in range(len(r)):
        if pmf[i] > -300 and pmf[i] < 300:
            SSD.append(r[i])
            PMF.append(pmf[i])
     
    #PMF = PMF - np.mean(PMF[-10:])
    


    spl = make_smoothing_spline(SSD, PMF, lam=0.0001)
    saveas = dictionary_convert_to_ua[file] + ".dat"
    np.savetxt("PP/"+ saveas, np.c_[SSD, spl(SSD)], delimiter=',')
    
    """
    # Plot the data
    plt.figure(figsize=(8, 6))
    plt.plot(SSD, PMF)
    plt.plot(SSD, spl(SSD), color='b')
    plt.xlabel('r (nm)', fontsize=14)
    plt.ylabel('Mean Force KbT', fontsize=14)
    plt.xlim(0, 2.5)
    plt.title('{} Potential Mean Force'.format(file), fontsize=16)
    plt.grid(True)
    #plt.legend()
    """
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 10))

    # RDF plot
    ax1.plot(r, RDF, color='r')
    ax1.set_ylabel(r"g(r) (k$_{B}$T)", fontsize=14)
    ax1.set_xlabel('SSD (nm)', fontsize=14)
    ax1.set_xlim(0, 2.5)
    ax1.set_title(f'{file} - RDF', fontsize=16)
    ax1.grid(True)

    # PMF plot
    ax2.plot(SSD, PMF, label='Raw PMF', alpha=0.6)
    ax2.plot(SSD, spl(SSD), color='b', label='Smooth PMF')
    ax2.set_xlabel('SSD (nm)', fontsize=14)
    ax2.set_ylabel(r"$U_{s}$ (k$_{B}$T)", fontsize=14)
    ax2.set_xlim(0, 2.5)
    ax2.set_title(f'{file} - PMF', fontsize=16)
    ax2.grid(True)
    ax2.legend()

    # Save as PDF
    fig.tight_layout()
    pdf_name = dictionary_convert_to_ua[file] + "_rdf_pmf.pdf"
    fig.savefig(f"{pdf_name}")
    plt.close(fig)
    
    
print(np.mean(radgyr))



/tmp/ipykernel_2242548/2654880738.py:49: RuntimeWarning: divide by zero encountered in log
  pmf = -np.log(RDF)
/tmp/ipykernel_2242548/2654880738.py:49: RuntimeWarning: divide by zero encountered in log
  pmf = -np.log(RDF)
/tmp/ipykernel_2242548/2654880738.py:49: RuntimeWarning: divide by zero encountered in log
  pmf = -np.log(RDF)
/tmp/ipykernel_2242548/2654880738.py:49: RuntimeWarning: divide by zero encountered in log
  pmf = -np.log(RDF)
/tmp/ipykernel_2242548/2654880738.py:49: RuntimeWarning: divide by zero encountered in log
  pmf = -np.log(RDF)
/tmp/ipykernel_2242548/2654880738.py:49: RuntimeWarning: divide by zero encountered in log
  pmf = -np.log(RDF)
/tmp/ipykernel_2242548/2654880738.py:49: RuntimeWarning: divide by zero encountered in log
  pmf = -np.log(RDF)
/tmp/ipykernel_2242548/2654880738.py:49: RuntimeWarning: divide by zero encountered in log
  pmf = -np.log(RDF)
/tmp/ipykernel_2242548/2654880738.py:49: RuntimeWarning: divide by zero encountered in log
  pmf = -np.l

0.30210387794321564
